<a href="https://colab.research.google.com/github/Sudarshinipilla/sudarshinifmml_fml1l2.inpynb/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
The accuracy of a model on a validation set can be affected by the percentage of data allocated to the validation set in various ways. Here’s a detailed look at how increasing or decreasing the size of the validation set impacts the accuracy:

### **1. Impact of Increasing the Percentage of the Validation Set**

When you increase the percentage of data allocated to the validation set, the following effects can be observed:

**1.1. **Reduced Training Data**:
   - **Less Training Data**: Increasing the validation set size means that less data is available for training. This reduction in training data can lead to poorer generalization if the remaining data is insufficient for the model to learn the underlying patterns effectively.
   - **Model Performance**: With less training data, the model might not capture all the necessary patterns, potentially leading to lower performance on both the training and validation sets.

**1.2. **Increased Validation Accuracy Variance**:
   - **Higher Variance**: A larger validation set provides a more stable and reliable estimate of the model’s performance, as it averages over a larger sample. However, the increased size can also lead to greater variance in accuracy if the validation set is not representative of the entire data distribution.
   - **More Accurate Estimation**: Generally, a larger validation set provides a more accurate estimate of the model’s performance on unseen data. This can be beneficial in understanding how the model might perform in a real-world scenario.

**1.3. **Model Tuning and Overfitting**:
   - **Better Hyperparameter Tuning**: With more data in the validation set, you can have a more reliable basis for tuning hyperparameters and selecting the best model configuration.
   - **Avoid Overfitting**: If the validation set is too small, there’s a risk that the model might overfit to it, leading to misleadingly high validation accuracy. A larger validation set reduces this risk by providing a more comprehensive evaluation.

### **2. Impact of Reducing the Percentage of the Validation Set**

When you decrease the percentage of data allocated to the validation set, the following effects can be observed:

**2.1. **Increased Training Data**:
   - **More Training Data**: Reducing the validation set size means more data is available for training. This can improve the model’s ability to learn from the data, potentially leading to better performance on the training set.
   - **Improved Model Performance**: With more data for training, the model might better capture the underlying patterns, potentially leading to better accuracy on unseen test data if the training is effective.

**2.2. **Less Reliable Validation Accuracy**:
   - **Higher Variability**: A smaller validation set might not be as representative of the overall data distribution. As a result, the accuracy reported might be less stable and more variable, providing a less reliable estimate of the model’s true performance.
   - **Overfitting Risks**: With less data in the validation set, there’s a higher risk of the model overfitting to the validation set, which can lead to overly optimistic performance estimates.

**2.3. **Hyperparameter Tuning and Model Evaluation**:
   - **Less Reliable Tuning**: A smaller validation set might make it harder to fine-tune hyperparameters effectively, as the feedback from the validation set might not be as comprehensive.
   - **Evaluation Challenges**: If the validation set is too small, it may not adequately represent the range of data the model will encounter in production, potentially leading to suboptimal model evaluations.

### **3. Balancing the Trade-off**

When deciding on the percentage of data to allocate to the validation set, it’s crucial to balance the trade-offs:

- **Typical Splits**: Common practice is to use a validation set size of around 10-20% of the total data, with the remaining data split between training and, if applicable, a separate test set.
- **Cross-Validation**: To mitigate the issues associated with validation set size, techniques like k-fold cross-validation can be used. In cross-validation, the dataset is split into k subsets, and the model is trained and validated k times, each time using a different subset as the validation set and the remaining as the training set. This approach provides a more robust estimate of model performance.

### **4. Conclusion**

The accuracy of the validation set is influenced by its size, with increasing the percentage typically providing a more stable estimate of performance but potentially reducing the amount of training data. Conversely, reducing the validation set size might improve the training phase by providing more data but can lead to less reliable validation results. Balancing the validation set size with the need for sufficient training data and employing techniques like cross-validation can help in obtaining a more accurate and reliable model evaluation.
2.
The size of the training and validation sets significantly impacts how well the validation set can predict accuracy on the test set. Here’s a detailed explanation of how these sizes influence prediction accuracy:

### **1. Impact of Training Set Size**

**1.1. **Training Data Quality**:
   - **More Data for Training**: A larger training set generally allows the model to learn more effectively, capturing more patterns and reducing overfitting. This can lead to a more robust model that performs better on unseen data, including the test set.
   - **Less Data for Training**: With a smaller training set, the model may struggle to generalize well, leading to overfitting on the training data and potentially poor performance on the test set.

**1.2. **Validation Set Accuracy Prediction**:
   - **Larger Training Set**: When the training set is large, the validation set provides a better approximation of the model’s performance because the model has had more exposure to diverse data. This typically makes the validation set a more reliable predictor of test set accuracy.
   - **Smaller Training Set**: With a smaller training set, the model might overfit or underfit, and the validation set may not accurately reflect how the model will perform on the test set. The validation accuracy might be optimistic or pessimistic depending on how well the model has generalized.

### **2. Impact of Validation Set Size**

**2.1. **Validation Set Representativeness**:
   - **Larger Validation Set**: A larger validation set provides a more comprehensive view of how well the model generalizes. This typically results in a more stable and reliable estimate of test set performance, as the validation set is more representative of the data distribution.
   - **Smaller Validation Set**: A smaller validation set may not adequately capture the diversity of the data distribution. This can lead to less stable and less accurate estimates of the test set performance. The validation set might either overestimate or underestimate the test set accuracy due to its limited representativeness.

**2.2. **Variance in Validation Performance**:
   - **Larger Validation Set**: With a larger validation set, the variance in the validation performance metrics (such as accuracy) tends to be lower, providing a more consistent estimate of how the model will perform on the test set.
   - **Smaller Validation Set**: Smaller validation sets can have higher variance in performance metrics, which can make it harder to accurately predict test set performance. The smaller sample might not reflect all possible variations in the test set.

### **3. Balancing Train and Validation Set Sizes**

**3.1. **Common Practice**:
   - **Typical Splits**: A common practice is to use around 70-80% of the data for training and 20-30% for validation. This balance helps ensure that the model has enough data to learn effectively while also providing a substantial validation set to estimate performance.
   - **Cross-Validation**: To improve the reliability of performance estimates, k-fold cross-validation is often used. This involves splitting the data into k subsets and iteratively using each subset as the validation set while the remaining k-1 subsets are used for training. This approach helps provide a more robust estimate of model performance.

**3.2. **Model Evaluation**:
   - **Test Set**: The test set, which is separate from both the training and validation sets, provides the final, unbiased estimate of the model’s performance. Properly setting aside a portion of data for testing ensures that the evaluation is not influenced by the choices made during training and validation.

### **4. Summary**

In summary, the sizes of the training and validation sets affect the accuracy of predictions on the test set in the following ways:

- **Larger Training Set**: Improves the model’s ability to generalize, leading to a validation set that better predicts test set performance.
- **Smaller Training Set**: Can lead to a model that doesn’t generalize well, causing the validation set to provide less reliable estimates of test set performance.

- **Larger Validation Set**: Offers a more accurate and stable estimate of how well the model will perform on the test set.
- **Smaller Validation Set**: May lead to higher variability and less accurate estimates of test set performance.

Balancing these sets and using techniques like cross-validation can help mitigate issues and provide a more reliable evaluation of model performance.
3.
Choosing a good percentage to reserve for the validation set involves balancing several factors to ensure that both the training and validation phases are effective. The goal is to allocate enough data to each set so that the model can be trained robustly while also having a sufficiently large validation set to provide a reliable estimate of model performance. Here’s a breakdown of typical recommendations and considerations for determining this percentage:

### **1. Common Practice for Validation Set Size**

**1.1. **Standard Splits**:
   - **80/20 Split**: A common practice is to use 80% of the data for training and 20% for validation. This split is a good starting point and is often used because it strikes a balance between having enough data for training while retaining a substantial portion for validation.
   - **70/30 Split**: Another common approach is to use 70% of the data for training and 30% for validation. This provides a larger validation set, which can be beneficial for models with complex architectures or when the data is more diverse.

**1.2. **Smaller Datasets**:
   - For smaller datasets, where every data point is valuable, it’s crucial to reserve enough data for validation without compromising the training process. In such cases, a split like 90/10 or even using techniques like cross-validation can be appropriate.

### **2. Factors Influencing the Percentage**

**2.1. **Dataset Size**:
   - **Large Datasets**: With large datasets, it’s often feasible to reserve a smaller percentage for validation (e.g., 10-20%) because even a smaller percentage still represents a substantial number of examples.
   - **Small Datasets**: With smaller datasets, you might need to reserve a higher percentage for validation (e.g., 20-30%) to ensure that the validation set is large enough to provide a reliable estimate of performance. Alternatively, you can use techniques like k-fold cross-validation to make the most of the available data.

**2.2. **Model Complexity**:
   - **Simple Models**: For less complex models or when the training data is sufficiently large, a smaller validation set percentage (e.g., 10-20%) may be adequate.
   - **Complex Models**: For more complex models that might require fine-tuning or when training data is limited, reserving a larger percentage for validation (e.g., 20-30%) might be beneficial to ensure reliable performance evaluation.

**2.3. **Evaluation Needs**:
   - **High-Stakes Applications**: In critical applications where performance is crucial, ensuring a larger validation set can provide more reliable estimates. Here, a 20-30% split might be appropriate.
   - **Iterative Development**: During iterative development and hyperparameter tuning, having a larger validation set can help in better evaluating model changes and avoiding overfitting to the validation data.

### **3. Cross-Validation as an Alternative**

**3.1. **K-Fold Cross-Validation**:
   - Instead of a single validation set, k-fold cross-validation divides the data into k subsets (or folds). The model is trained k times, each time using k-1 folds for training and the remaining fold for validation. This approach helps in obtaining a more robust estimate of model performance and makes better use of the data.
   - Typical values for k range from 5 to 10, depending on the dataset size and computational resources. For instance, with 10-fold cross-validation, each subset serves as a validation set 10% of the time, providing a balanced approach to model evaluation.

### **4. Summary and Recommendations**

- **General Recommendation**: For many practical scenarios, reserving 20% of the data for the validation set (i.e., an 80/20 split) is a good starting point. It provides a reasonable balance between training and validation data.
- **For Smaller Datasets**: Consider reserving 20-30% for validation or using cross-validation to maximize data usage.
- **For Larger Datasets**: A 10-20% validation set might suffice, given the large number of samples available for training.
- **Model Complexity and Application**: Adjust the percentage based on model complexity and application requirements, ensuring that the validation set size provides reliable performance estimates.

In conclusion, the choice of the percentage for the validation set should consider the size of the dataset, the complexity of the model, and the specific needs of the application. Balancing these factors ensures that the model is well-trained and that the validation set provides an accurate estimate of performance on unseen data.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.